In [1]:
print("Hello|")

Hello|


In [3]:
pip install -U langchain-community


  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.5.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached marshmallow-3.22.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 357.1 kB/s eta 0:00:06
   -------- ------------------------------- 0.5/2.4 MB 357.1 kB/s eta 0:00:06
  

In [4]:
pip install -U langchain pinecone-client ctransformers


  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.9 MB 6.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.4/9.9 MB 6.1 MB/s eta 0:00:02
   -------------- ------------------------- 3.7/9.9 MB 6.1 MB/s eta 0:00:02
   ------------------- -------------------- 4.7/9.9 MB 6.1 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.9 MB 5.8 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/9.9 MB 5.7 MB/s eta 0:00:01
   -------------------------------- ------- 8.1/9.9 MB 5.7 MB/s eta 0:00:01
   ------------------------------------- -- 9.2/9.9 MB 5.5 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.9 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 5.0 MB/s eta 0:00:00


In [23]:
pip install pymupdf

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   --------- ------------------------------ 0.8/3.2 MB 4.8 MB/s eta 0:00:01
   ---------------------- ----------------- 1.8/3.2 MB 5.3 MB/s eta 0:00:01
   -------------------------------- ------- 2.6/3.2 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.2 MB 5.6 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/13.2 MB 5.9 MB/s eta 0:00:02
   ------- -------------------------------- 2.6/13.2 MB 4.0 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/13.2 MB 3.7 MB/s eta 0:00:03
   ----------- ---------------------------- 3.9/13.2 MB 3.7 MB/s eta 0:00:03
   --------------- ------------------------ 5.0/13.2 MB 3.9 MB/s eta 0:00:03
   ----------------- ---------------------- 5.8/13.2 MB 4.0 MB/s eta 0:00:02
   ---------------

In [43]:
pip install -U langchain-huggingface


Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install -U pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [7]:
from tqdm.autonotebook import tqdm

In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers

In [28]:
PINECONE_API_KEY = "c6ce942a-5f94-4668-bc1d-2ebae7713e40"

In [5]:
#Extract data from PDF files
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyMuPDFLoader)
    documents = loader.load()

    return documents



In [6]:
extracted_data = load_pdf("data/")

d:\Code\Medical chatbot\Medical-Chatbot-using-llama\medical\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:299: UserWarning: Warning: Empty content on page 0 of document data\Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf
  warnings.warn(


In [7]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunks: ", len(text_chunks))

length of my chunks:  5779


In [9]:
#download embeddings model
def download_huggingface_model():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_huggingface_model()

C:\Users\abhis\AppData\Local\Temp\ipykernel_19856\192741830.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2")
d:\Code\Medical chatbot\Medical-Chatbot-using-llama\medical\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result = embeddings.embed_query("Hello, how are you?")
print("length of my query result: ", len(query_result))

length of my query result:  384


In [13]:
query_result

[0.5421559810638428,
 0.18908195197582245,
 0.8348346948623657,
 0.019833765923976898,
 -0.8182635307312012,
 -0.5520739555358887,
 0.5031420588493347,
 0.11975957453250885,
 -0.2559368312358856,
 0.2397879958152771,
 0.06577672064304352,
 -0.4289025068283081,
 -0.24317021667957306,
 -0.14757925271987915,
 0.21157336235046387,
 0.27461254596710205,
 0.22511503100395203,
 -0.4708331525325775,
 -0.8574920296669006,
 -0.02177487686276436,
 -0.44853755831718445,
 -0.03448502719402313,
 -0.11906781792640686,
 -0.015861667692661285,
 0.34434258937835693,
 -0.016430480405688286,
 0.012980563566088676,
 0.6993362307548523,
 0.033979810774326324,
 0.05523053556680679,
 -0.1583712249994278,
 -0.16747117042541504,
 -0.10531020164489746,
 0.15381643176078796,
 0.1834600418806076,
 0.3758128881454468,
 0.10488469153642654,
 0.10315363109111786,
 -0.18420681357383728,
 0.17847377061843872,
 0.00675179623067379,
 -0.3362417221069336,
 -0.04654860123991966,
 -0.07281830161809921,
 0.3359984755516052,


In [56]:
pinecone.init(api_key=PINECONE_API_KEY,
              environment=ServerlessSpec(cloud="aws", region="us-east-1"))
index_name = "geography-chatbot"

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

